# Dev 07 - Wind-Turbine Location

In [7]:
import csv
import numpy as np

In [14]:
def wtloc_from_csv(filename):
    data = []
    with open(filename) as file:
        reader = csv.reader(file, delimiter=',')
        for k, row in enumerate(reader):
            data.append(row)
    return np.array(data, dtype=np.single)

In [15]:
loc = wtloc_from_csv('wtloc.csv')

In [16]:
loc

array([[  1.   ,   5.333, 354.32 ],
       [  2.   ,   5.307, 351.26 ],
       [  3.   ,   5.606, 350.12 ],
       [  4.   ,   5.874, 350.89 ],
       [  5.   ,   6.108, 350.06 ],
       [  6.   ,   6.749, 348.95 ],
       [  7.   ,   6.976, 348.99 ],
       [  8.   ,  11.799, 342.54 ],
       [  9.   ,  12.073, 342.17 ],
       [ 10.   ,  12.347, 341.9  ],
       [ 11.   ,  12.609, 341.43 ],
       [ 12.   ,  12.895, 340.91 ],
       [ 13.   ,  13.16 , 340.53 ],
       [ 14.   ,  13.421, 340.01 ],
       [ 15.   ,  13.722, 339.6  ],
       [ 16.   ,  13.992, 339.25 ],
       [ 17.   ,  14.265, 338.81 ],
       [ 18.   ,  14.528, 338.45 ],
       [ 19.   ,  14.794, 338.06 ],
       [ 20.   ,  15.057, 337.67 ],
       [ 21.   ,  15.316, 337.29 ],
       [ 22.   ,  15.573, 336.92 ],
       [ 23.   ,  15.846, 336.64 ],
       [ 24.   ,  16.136, 336.64 ],
       [ 25.   ,   5.531, 170.6  ],
       [ 26.   ,   5.17 , 170.44 ],
       [ 27.   ,   5.591, 167.29 ]], dtype=float32)